In [1]:
library(broom)
library(nnet)
library(forcats)
library(stargazer)
library(data.table)
library(lme4)
library(lmerTest)
library(car)



Please cite as: 

 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 

Loading required package: Matrix
Warning message:
“package ‘Matrix’ was built under R version 3.6.2”Warning message:
“package ‘lmerTest’ was built under R version 3.6.2”
Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step

Loading required package: carData
Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4


In [221]:
data = read.table('/shared/2/projects/framing/results/full_datasheet_11-13-20.tsv',header=TRUE,sep='\t')
#data = read.table('/shared/2/projects/framing/results/data_5_percent_11-13-20.tsv',header=TRUE,sep='\t')

In [222]:
data$id_str <- NULL
data$X <- NULL
data$Unnamed..0 <- NULL
data$op_ideology <- NULL
data$opposed_ideology <- NULL
data$Hero <- NULL
data$Victim <- NULL
data$Threat <- NULL
data <- na.omit(data)
data <- data[data$country=='US', ]


In [223]:
cols <- cbind(colnames(data))
frames <- cols[1:27]
features <- cols[28:length(cols)]
factor_features <- cbind('country','has_hashtag','has_mention','has_url','is_quote_status','is_reply',
                         'is_verified','date','month','year')
data[,frames] <-lapply(data[,frames],as.factor)
data[,factor_features] <-lapply(data[,factor_features],as.factor)

In [273]:
frames
frames_generic <- c(frames[1:4],frames[6:8],frames[12:18])
frames_narrative <- c(frames[5],frames[19])
frames_specific <- c(frames[9:11],frames[20:27])


[1] "Capacity.and.Resources"                   
 [2] "Crime.and.Punishment"                     
 [3] "Cultural.Identity"                        
 [4] "Economic"                                 
 [5] "Episodic"                                 
 [6] "External.Regulation.and.Reputation"       
 [7] "Fairness.and.Equality"                    
 [8] "Health.and.Safety"                        
 [9] "Hero..Cultural.Diversity"                 
[10] "Hero..Integration"                        
[11] "Hero..Worker"                             
[12] "Legality..Constitutionality..Jurisdiction"
[13] "Morality.and.Ethics"                      
[14] "Policy.Prescription.and.Evaluation"       
[15] "Political.Factors.and.Implications"       
[16] "Public.Sentiment"                         
[17] "Quality.of.Life"                          
[18] "Security.and.Defense"                     
[19] "Thematic"                                 
[20] "Threat..Fiscal"                           
[21] "Threat..Jobs"                             
[22] "Threat..National.Cohesion"                
[23] "Threat..Public.Order"                     
[24] "Victim..Discrimination"                   
[25] "Victim..Global.Economy"                   
[26] "Victim..Humanitarian"                     
[27] "Victim..War"

In [274]:
fav_features <- c('date','month','year',
                      'has_hashtag','has_mention','has_url',
                     'is_quote_status','is_reply','is_verified',
                     'log_chars','log_followers','log_following','log_statuses',
                      'ideology','log_favorites')
rt_features <- c('date','month','year',
                      'has_hashtag','has_mention','has_url',
                     'is_quote_status','is_reply','is_verified',
                     'log_chars','log_followers','log_following','log_statuses',
                      'ideology','log_retweets')

In [303]:
fav_df <- subset(data, select = c(frames_generic,fav_features))
rt_df <- subset(data, select = c(frames_generic,rt_features))

In [304]:
fav_formula <- as.formula("log_favorites ~ . + (1 | year/month/date) - date - month - year")

In [305]:
rt_formula <- as.formula("log_retweets ~ . + (1 | year/month/date) - date - month - year")

In [306]:
# Analysis 3: Predict Favorites from Frames (Linear Regression)
fav_model = lmer(fav_formula, data = fav_df,control=lmerControl(calc.derivs=FALSE))

In [307]:
# Analysis 4: Predict Retweets from Frames (Linear Regression)
rt_model = lmer(rt_formula, data = rt_df,control=lmerControl(calc.derivs=FALSE))

In [308]:
fav_orig_p <- summary(fav_model)$coefficients[,"Pr(>|t|)"]
rt_orig_p <- summary(rt_model)$coefficients[,"Pr(>|t|)"]

In [309]:
fav_corrected_p <- p.adjust(fav_orig_p, method = "holm", n = length(fav_orig_p))
rt_corrected_p <- p.adjust(rt_orig_p, method = "holm", n = length(rt_orig_p))

In [310]:
class(rt_model) <- "lmerMod"
class(fav_model) <- "lmerMod"

In [311]:
fav_results <- tidy(fav_model)
rt_results <- tidy(rt_model)
fav_results$p.corrected <- c(fav_corrected_p, rep(NA, nrow(fav_results)-length(fav_corrected_p)))
rt_results$p.corrected <- c(rt_corrected_p,rep(NA,nrow(rt_results)-length(rt_corrected_p)))


In [312]:
fav_filename = paste('/shared/2/projects/framing/results/frame_setting/fav_model_3-31-21_generic.Rdata',sep='')
save(fav_model, file=fav_filename)

In [313]:
rt_filename = paste('/shared/2/projects/framing/results/frame_setting/rt_model_3-31-21_generic.Rdata',sep='')
save(rt_model, file=rt_filename)

In [314]:
results_path <- "/shared/2/projects/framing/results/frame_setting/"
write.table(fav_results,paste(results_path,'favorites_3-31-21_generic.tsv',sep=''),sep='\t')
write.table(rt_results,paste(results_path,'retweets_3-31-21_generic.tsv',sep=''),sep='\t',)

In [315]:
stargazer(fav_model,rt_model,no.space=TRUE,single.row=TRUE,
          out=paste(results_path,'fav_rt_3-31-21_generic.tex',sep=''),
          p.auto=FALSE,
          p=list(c(fav_corrected_p,rt_corrected_p)),
          star.cutoffs=c(.05,.01,.005))


% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Thu, Apr 01, 2021 - 04:48:08 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lcc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{2}{c}{\textit{Dependent variable:}} \\ 
\cline{2-3} 
\\[-1.8ex] & log\_favorites & log\_retweets \\ 
\\[-1.8ex] & (1) & (2)\\ 
\hline \\[-1.8ex] 
 Capacity.and.Resources1 & $-$0.028$^{***}$ (0.003) & $-$0.011$^{***}$ (0.003) \\ 
  Crime.and.Punishment1 & $-$0.010$^{***}$ (0.002) & 0.030$^{***}$ (0.001) \\ 
  Cultural.Identity1 & 0.029 (0.002) & 0.001 (0.002) \\ 
  Economic1 & 0.008$^{***}$ (0.002) & 0.017$^{***}$ (0.002) \\ 
  External.Regulation.and.Reputation1 & $-$0.024 (0.004) & $-$0.003 (0.003) \\ 
  Fairness.and.Equality1 & 0.048$^{***}$ (0.002) & 0.032$^{***}$ (0.002) \\ 
  Health.and.Safety1 & $-$0.006$^{***}$ (0.002) & 0.026$^{***}$ (0.002) \\ 
  Legality..Constitutional

In [316]:
stargazer(fav_model,rt_model,no.space=TRUE,single.row=TRUE,type='text',
          p.auto=FALSE,
          p=list(c(fav_corrected_p,rt_corrected_p)),
          star.cutoffs=c(.05,.01,.005))


                                                    Dependent variable:         
                                           -------------------------------------
                                             log_favorites       log_retweets   
                                                  (1)                (2)        
--------------------------------------------------------------------------------
Capacity.and.Resources1                    -0.028*** (0.003)  -0.011*** (0.003) 
Crime.and.Punishment1                      -0.010*** (0.002)   0.030*** (0.001) 
Cultural.Identity1                           0.029 (0.002)      0.001 (0.002)   
Economic1                                   0.008*** (0.002)   0.017*** (0.002) 
External.Regulation.and.Reputation1          -0.024 (0.004)     -0.003 (0.003)  
Fairness.and.Equality1                      0.048*** (0.002)   0.032*** (0.002) 
Health.and.Safety1                         -0.006*** (0.002)   0.026*** (0.002) 
Legality..Constitutionality

In [187]:
nrow(fav_data)

[1] 2293954

In [191]:
length(fav_data['ideology'][is.na(fav_data['ideology'])])

[1] 613789

In [193]:
summary(lm(log_favorites ~ ideology,data=fav_data))


Call:
lm(formula = log_favorites ~ ideology, data = fav_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.7412 -0.5932 -0.4761  0.2184 11.4250 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.6171212  0.0010310  598.59   <2e-16 ***
ideology    -0.0524281  0.0005457  -96.08   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.022 on 1262324 degrees of freedom
  (1031628 observations deleted due to missingness)
Multiple R-squared:  0.00726,	Adjusted R-squared:  0.007259 
F-statistic:  9232 on 1 and 1262324 DF,  p-value: < 2.2e-16


In [205]:
nrow(na.omit(fav_data))

[1] 576288

In [217]:
nrow(fav_data)

[1] 1262326

In [229]:
nrow(na.omit(fav_df))

[1] 1262326

In [230]:
nrow(na.omit(rt_df))

[1] 1262326

In [318]:
summary(fav_model)


Correlation matrix not shown by default, as p = 26 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it



Linear mixed model fit by REML ['lmerMod']
Formula: fav_formula
   Data: fav_df
Control: lmerControl(calc.derivs = FALSE)

REML criterion at convergence: 3226658

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.0708 -0.5823 -0.2089  0.3246 11.8799 

Random effects:
 Groups            Name        Variance  Std.Dev.
 date:(month:year) (Intercept) 0.0005722 0.02392 
 month:year        (Intercept) 0.0011780 0.03432 
 year              (Intercept) 0.0018345 0.04283 
 Residual                      0.7539513 0.86830 
Number of obs: 1262326, groups:  
date:(month:year), 616; month:year, 24; year, 3

Fixed effects:
                                             Estimate Std. Error  t value
(Intercept)                                -0.7666740  0.0323010  -23.735
Capacity.and.Resources1                    -0.0275600  0.0034227   -8.052
Crime.and.Punishment1                      -0.0096697  0.0017526   -5.517
Cultural.Identity1                          0.0288000  0.0024060   11.970
E